<a href="https://colab.research.google.com/github/jinbac/tensor/blob/main/Plantnet_inceptionv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dataset URL
# https://zenodo.org/record/4726653 
!wget https://zenodo.org/record/4726653/files/plantnet_300K.zip?download=1

# Get DATSET from here
# https://gitlab.inria.fr/cgarcin/plantnet_dataset

--2021-08-16 01:53:51--  https://zenodo.org/record/4726653/files/plantnet_300K.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31654430714 (29G) [application/octet-stream]
Saving to: ‘plantnet_300K.zip?download=1’

plantnet_300K.zip?d 100%[===================>]  29.48G  24.1MB/s    in 23m 17s 

2021-08-16 02:17:09 (21.6 MB/s) - ‘plantnet_300K.zip?download=1’ saved [31654430714/31654430714]



## Mount Google Drive

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Tensorflow and other things

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pprint
import json
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score


import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image

## Check you are running on a TPU

In [3]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

BaseException: ignored

## Version Notice
This code assumes you are using tensorflow 2.0 or later and python 3.6 or later

In [4]:
print("Tensorflow version is " + tf.__version__)
print("Numpy version is " + np.__version__)

Tensorflow version is 2.6.0
Numpy version is 1.19.5


In [5]:
!pip install git+https://github.com/tensorflow/docs


  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-0otyrau2
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-0otyrau2
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0935fa8e3521a702021ac865fcbb0bf020e50b458_-py3-none-any.whl size=154539 sha256=3871678d609256727d1419e2266c78dced0a750307b4ae07f84b468948bd0e50
  Stored in directory: /tmp/pip-ephem-wheel-cache-oi4gfwz4/wheels/cc/c4/d8/5341e93b6376c5c929c49469fce21155eb69cef1a4da4ce32c
Failed to build tensorflow-docs
    Running setup.py install for tensorflow-docs ... done
  DEPRECATION: tensorflow-docs was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [6]:
import tensorflow_docs as tdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [7]:
# Unzips files to google colab
!unzip /content/drive/MyDrive/plantnet/plantnet_300K.zip

Streaming output truncated to the last 5000 lines.
  inflating: plantnet_300K/images_test/1363764/36292b7a83e33de3764ed68fc16543bad5bb07db.jpg  
  inflating: plantnet_300K/images_test/1363764/069f465bf6f0757669985a67789f7fb3a8ab41cc.jpg  
  inflating: plantnet_300K/images_test/1363764/18a7639b314d0f10d985f4872b00d7d5c24982a2.jpg  
  inflating: plantnet_300K/images_test/1363764/db66ae8da58d4b5485324a11b6fa7472cfc07bfb.jpg  
  inflating: plantnet_300K/images_test/1363764/e462607ec30224bb48cb7d286a2436275c1bc01e.jpg  
  inflating: plantnet_300K/images_test/1363764/1d16243c59cced47cd91990b2d1b83f5533914be.jpg  
  inflating: plantnet_300K/images_test/1363764/c947804a38eaa45720b7691fa8560416f621eb12.jpg  
  inflating: plantnet_300K/images_test/1363764/357afa09d847115ef35e14164e4cc5011f9d8f35.jpg  
  inflating: plantnet_300K/images_test/1363764/48db1f52629c8c3bae053b111ea6f42e545283fe.jpg  
  inflating: plantnet_300K/images_test/1363764/cd4a080a296494feccf9505a3fa9b73ab0a633df.jpg  
  inflati

In [8]:
# Check file unzipped properly and is 31GB
!du -hs plantnet_300K/

31G	plantnet_300K/


In [9]:
ls plantnet_300K/

images_test/   images_val/                 plantnet300K_species_names.json
images_train/  plantnet300K_metadata.json  README


In [10]:
 # Check species classes vs folder names
!cat plantnet_300K/plantnet300K_species_names.json

{"1355868": "Lactuca_virosa", "1355920": "Pelargonium_capitatum", "1355932": "Pelargonium_graveolens", "1355936": "Cirsium_arvense", "1355937": "Cirsium_vulgare", "1355955": "Pelargonium_odoratissimum", "1355959": "Pelargonium_peltatum", "1355961": "Pelargonium_quercifolium", "1355978": "Pelargonium_zonale", "1355990": "Mercurialis_annua", "1356003": "Phyllanthus_tenellus", "1356022": "Hypericum_perforatum", "1356037": "Egeria_densa", "1356055": "Ibicella_lutea", "1356075": "Tradescantia_fluminensis", "1356076": "Tradescantia_zebrina", "1356111": "Lamium_amplexicaule", "1356126": "Lavandula_dentata", "1356138": "Striga_asiatica", "1356257": "Melilotus_albus", "1356278": "Trifolium_fragiferum", "1356279": "Asystasia_gangetica", "1356309": "Nymphaea_nouchali", "1356379": "Dryopteris_aemula", "1356380": "Dryopteris_affinis", "1356382": "Dryopteris_filix-mas", "1356420": "Nephrolepis_cordifolia", "1356421": "Nephrolepis_exaltata", "1356428": "Osmunda_regalis", "1356469": "Achyranthes_asper

## Create Datasets for training process

In [11]:
seed = 123
image_size = (224, 224)
batch_size = 32

train_folder = "plantnet_300K/images_train/"
validation_folder = "plantnet_300K/images_val/"
test_folder = "plantnet_300K/images_test/"

# train_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
#     "plantnet_300K/images_train/", labels='inferred', label_mode='categorical',
#     class_names=None, color_mode='rgb', batch_size=batch_size, image_size=image_size, shuffle=True, seed=seed,
#     interpolation='bilinear', follow_links=False, smart_resize=True
# )
# val_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
#     "plantnet_300K/images_val/", labels='inferred', label_mode='categorical',
#     class_names=None, color_mode='rgb', batch_size=batch_size, image_size=image_size,
#      shuffle=True, seed=seed,
#     interpolation='bilinear', follow_links=False, smart_resize=True
# )

# test_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
#     "plantnet_300K/images_test/", labels='inferred', label_mode='categorical',
#     class_names=None, color_mode='rgb', batch_size=batch_size, image_size=image_size,
#      shuffle=True, seed=seed,
#     interpolation='bilinear', follow_links=False, smart_resize=True
# )

# By default flow_from_directory shuffles data
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=45,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range=(0.3,0.9),
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
        validation_folder,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 243916 images belonging to 1081 classes.
Found 31118 images belonging to 1081 classes.


In [13]:
# import numpy as np
     #train_dataset_raw.shuffle(buffer_size=1024)
# it = iter(train_dataset_raw)
# next(it)

## Preprocess Datasets

In [12]:
# AUTO = tf.data.experimental.AUTOTUNE
# PREPROCESS = True
# BATCH_SIZE = 256

# if PREPROCESS:
#   # Resize all Images
#   @tf.function
#   def process_dataset(image, label):
#     # image = tf.image.resize(image, (224, 224))
#     # image = preprocess_input(image)
#     image = image/127.0 - 1 #normalize around -1 to +1
#     return(image, label)

#   train_dataset = train_dataset_raw.map(process_dataset, num_parallel_calls=AUTO)
#   val_dataset = val_dataset_raw.map(process_dataset, num_parallel_calls=AUTO)
#   test_dataset = test_dataset_raw.map(process_dataset, num_parallel_calls=AUTO)

# else:
#   train_dataset = train_dataset_raw
#   val_dataset = val_dataset_raw
#   test_dataset = test_dataset_raw


train_dataset = train_generator
val_dataset = validation_generator

## Create Model Base from MobileNetV2 without top layers


In [13]:
# Mobilenet was originally trained with -1 to +1 values, Normalizer sets images within this range so pretrained weights work properly
# normalizer = tf.keras.layers.experimental.preprocessing.Normalization(mean=0, variance=1)
# inputs = tf.keras.Input(shape=(224,224,3))
# input_layer = normalizer(inputs)
# normalizer.adapt(train_dataset_raw.batch(128))


base_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(224,224,3), pooling=None, classes=1081,
    classifier_activation='softmax'
)

# Combine input layer with base model
# full_model = Model(inputs=[inputs], outputs=[base_model.output])

87924736/87910968 [==============================] - 1s 0us/step


In [41]:
#it = iter(test_dataset_raw)

In [42]:
#batch_images = next(it)[0]

In [43]:
#single_image = batch_images[0]

In [ ]:
#single_image

In [ ]:
#preprocess_input(single_image)

In [ ]:
   #it = iter(test_dataset)
   #single_image_norm = next(it)[0][0]
   #single_image_norm
# x = (single_image_norm - preprocess_input(single_image))
# print(x)

## Add Top layer to base model

In [14]:
# normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
# normalizer.adapt(train_dataset_raw.batch(128))

# inputs = tf.keras.Input(shape=(224,224,3))
# x = normalizer(inputs)
# x = tf.keras.backend.function([x, tf.keras.backend.learning_phase()], base_model.input) 
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation="relu")(x)
predictions = Dense(1081, activation="softmax")(x)

## Finish creating complete model



In [15]:
# model = Model(inputs = inputs, outputs = predictions)
model = Model(inputs = base_model.input, outputs = predictions)

## Freeze the model middle layers

In [16]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
# for layer in base_model.layers:
#     layer.trainable = False
base_model.trainable = False

In [17]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy' , metrics= ["accuracy"])

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

In [19]:
# Callbacks

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from datetime import datetime 

current_time = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
gdrive_dir = "/content/drive/MyDrive/saved_models/"
filepath = "{}plantNet_20210818{}.h5".format(gdrive_dir, "{}".format("_Trainable" if base_model.trainable else ""))    # filepath should not pull something if traing from scratch 

# Try loading model at filepath to continue training
# If no file exists then it creates a new one
try:
  model = keras.models.load_model(filepath)
  print("Continuing training from checkpoint")
except:
  print("No keras model found at {}, will start training from scratch.".format(filepath))
  filepath = "{0}plantNet_{1}{2}.h5".format(gdrive_dir, datetime.now().strftime('%Y%m%d'), "_Trainable" if base_model.trainable else "")
  print("New model filename is {}".format(filepath))


# Save model only when validation accuracy increases
# checkpoint = ModelCheckpoint(filepath, monitor="val_acc", verbose=1, save_best_only=True, mode="max")
checkpoint = ModelCheckpoint(filepath, save_best_only=False, save_weights_only=False)

# Tensorboard callback
logdir = gdrive_dir + "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Stop training model if validation loss does not improve for 3 epochs
earlyStopping = EarlyStopping(monitor="val_loss", patience=3, verbose=1)

callbacks_list = [checkpoint, earlyStopping]

No keras model found at /content/drive/MyDrive/saved_models/plantNet_20210809.h5, will start training from scratch.
New model filename is /content/drive/MyDrive/saved_models/plantNet_20210818.h5


Keep logs in gdrive

In [20]:
gdrive_dir + "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

'/content/drive/MyDrive/saved_models/logs/20210818-233139'

## Train Model

In [ ]:
epochs = 5
steps_per_epoch = None
# val_dataset = None
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, callbacks=callbacks_list, steps_per_epoch=steps_per_epoch)



Epoch 1/5
7623/7623 [==============================] - 4847s 633ms/step - loss: 4.0884 - accuracy: 0.2568 - val_loss: 4.1532 - val_accuracy: 0.2672
Epoch 2/5
 243/7623 [..............................] - ETA: 1:14:55 - loss: 4.2671 - accuracy: 0.2501

'Trainable'

In [ ]:
ls

drive/  model.png  plantnet_300K/  sample_data/
